# Memory in Autogen

## How much is the fever - Fahrenheit
## What's the temperature in Celsius

AgentChat provides a [`Memory`](https://microsoft.github.io/autogen/stable//reference/python/autogen_core.memory.html#autogen_core.memory.Memory "autogen_core.memory.Memory") protocol that can be extended to provide this functionality. The key methods are `query`, `update_context`, `add`, `clear`, and `close`.

-   `add`: add new entries to the memory store

-   `query`: retrieve relevant information from the memory store

-   `update_context`: mutate an agent's internal `model_context` by adding the retrieved information (used in the [`AssistantAgent`](https://microsoft.github.io/autogen/stable//reference/python/autogen_agentchat.agents.html#autogen_agentchat.agents.AssistantAgent "autogen_agentchat.agents.AssistantAgent") class)

-   `clear`: clear all entries from the memory store

-   `close`: clean up any resources used by the memory store

In [6]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GOOGLE_GEMINI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console

from autogen_core.memory import ListMemory,MemoryContent,MemoryMimeType


# Create a memory for the agent
user_memory = ListMemory()

await user_memory.add(MemoryContent(content='Weather should be in metric units',mime_type=MemoryMimeType.TEXT))
await user_memory.add(MemoryContent(content="Meal recipe should be vegan",mime_type=MemoryMimeType.TEXT))


In [8]:
async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 °F and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 °C and Sunny."
    else:
        return f"Sorry, I don't know the weather in {city}."

In [9]:
assistant_agent = AssistantAgent(
    name = 'Assistant_agent',
    model_client=model_client,
    memory = [user_memory],
    tools= [get_weather],
    system_message="""You are a helpful assistant. You can answer questions about the and also use tools wherenver neccessary."""
)

In [10]:
stream = assistant_agent.run_stream(task = "What is the weather in New York?")
await(Console(stream))

---------- TextMessage (user) ----------
What is the weather in New York?
---------- MemoryQueryEvent (Assistant_agent) ----------
[MemoryContent(content='Weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe should be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- ToolCallRequestEvent (Assistant_agent) ----------
[FunctionCall(id='call_t5sPwv0cphJJ890SBLj4l28V', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
---------- ToolCallExecutionEvent (Assistant_agent) ----------
[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_t5sPwv0cphJJ890SBLj4l28V', is_error=False)]
---------- ToolCallSummaryMessage (Assistant_agent) ----------
The weather in New York is 23 °C and Sunny.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What is the weather in New York?', type='TextMessage'), MemoryQueryEvent(source='Assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='Weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe should be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), ToolCallRequestEvent(source='Assistant_agent', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=19), metadata={}, content=[FunctionCall(id='call_t5sPwv0cphJJ890SBLj4l28V', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='Assistant_agent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_t5sPwv0cphJJ890

In [11]:
await assistant_agent.model_context.get_messages()

[UserMessage(content='What is the weather in New York?', source='user', type='UserMessage'),
 SystemMessage(content='\nRelevant memory content (in chronological order):\n1. Weather should be in metric units\n2. Meal recipe should be vegan\n', type='SystemMessage'),
 AssistantMessage(content=[FunctionCall(id='call_t5sPwv0cphJJ890SBLj4l28V', arguments='{"city":"New York","units":"metric"}', name='get_weather')], thought=None, source='Assistant_agent', type='AssistantMessage'),
 FunctionExecutionResultMessage(content=[FunctionExecutionResult(content='The weather in New York is 23 °C and Sunny.', name='get_weather', call_id='call_t5sPwv0cphJJ890SBLj4l28V', is_error=False)], type='FunctionExecutionResultMessage')]

In [12]:
stream = assistant_agent.run_stream(task = "Can you give me a soup recipe ?")
await(Console(stream))

---------- TextMessage (user) ----------
Can you give me a soup recipe ?
---------- MemoryQueryEvent (Assistant_agent) ----------
[MemoryContent(content='Weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe should be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)]
---------- TextMessage (Assistant_agent) ----------
Here's a delicious vegan soup recipe for you:

**Vegan Lentil Soup**

**Ingredients:**
- 1 tablespoon olive oil
- 1 onion, chopped
- 2 carrots, diced
- 2 celery stalks, diced
- 3 cloves garlic, minced
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon smoked paprika
- 1 cup dried lentils, rinsed
- 1 can (14-ounce) diced tomatoes
- 4 cups vegetable broth
- 2 cups water
- Salt and pepper to taste
- 2 cups kale or spinach, chopped
- 1 tablespoon lemon juice

**Instructions:**
1. In a large pot, heat olive oil over medium heat. Add onions, carrots, and celery

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Can you give me a soup recipe ?', type='TextMessage'), MemoryQueryEvent(source='Assistant_agent', models_usage=None, metadata={}, content=[MemoryContent(content='Weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None), MemoryContent(content='Meal recipe should be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None)], type='MemoryQueryEvent'), TextMessage(source='Assistant_agent', models_usage=RequestUsage(prompt_tokens=204, completion_tokens=290), metadata={}, content="Here's a delicious vegan soup recipe for you:\n\n**Vegan Lentil Soup**\n\n**Ingredients:**\n- 1 tablespoon olive oil\n- 1 onion, chopped\n- 2 carrots, diced\n- 2 celery stalks, diced\n- 3 cloves garlic, minced\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon smoked paprika\n- 1 cup dried lentils, rinsed\n- 1 can (14-ounce) diced tomatoes\n- 4 cups veget